# Description
This python script performs continuum removal (CR) on spectra of binary mixtures and endmembers. \
In addition, on each isolated band it computes band area (BA), band depth (BD) and band centre (BC). 


In [2]:
#import
import numpy as np #numeric python
import matplotlib.pyplot as plt #plots
import matplotlib.ticker as ticker #set ticks spacing 
import seaborn as sns
from find_nearest import find_nearest #to find nearest value in a numpy array 


#use %matplotlib to display figures on screen 
%matplotlib 

Using matplotlib backend: <object object at 0x113ed40b0>


# Read and plot data 

In [2]:
path = 'Data/'
endmember1_name = 'hexa' #put the clay/sulfate endmember name here. Other endmembers are: NAu-1, NAu-2 and SM1200H
endmember2_name = 'FV7' #basalt endmember
mix_ls = ['10', '20', '30', '40','50', '60', '70', '80', '90'] #relative abundance (%)
means_ls = [] #list containing mean spectra 
labels_ls = [] #list with labels

for i,name in enumerate(mix_ls):

    #right combination to retrieve the filenames: np.loadtxt(path+endmember1_name + '-' + mix_ls[i] + '_' + endmember2_name + '-' + mix_ls[-i-1]...) for SM1200H
    #or: np.loadtxt(endmember1_name + '_' + mix_ls[i] + '_' + endmember2_name + '_' + mix_ls[-i-1]...) for NAu-1, NAu-2, and Hexa

    s0 = np.loadtxt(path+endmember1_name+'_'+mix_ls[i]+'_'+endmember2_name+'_'+mix_ls[-i-1]+'_00000.asd.rts.txt', delimiter='\t', skiprows = 1)
    s1 = np.loadtxt(path+endmember1_name+'_'+mix_ls[i]+'_'+endmember2_name+'_'+mix_ls[-i-1]+'_00001.asd.rts.txt', delimiter='\t', skiprows = 1)
    s2 = np.loadtxt(path+endmember1_name+'_'+mix_ls[i]+'_'+endmember2_name+'_'+mix_ls[-i-1]+'_00002.asd.rts.txt', delimiter='\t', skiprows = 1)

    s_mean = (s0[:,1]+s1[:,1]+s2[:,1])/3

    means_ls.append(s_mean) #adding the spectrum to a list
    labels_ls.append(endmember1_name+'-'+mix_ls[i]+'_'+endmember2_name+'-'+mix_ls[-i-1]) #adding the label to a list

wvl = s0[:,0]*0.001 #wavelength in microns
spectra = np.array(means_ls).T #all the spectra go from the list to this numpy array
print('Spectra shape:',spectra.shape) #checking shape

Spectra shape: (2151, 9)


In [3]:
#use %matplotlib to display figures on screen 
%matplotlib 

#plot (with or without offset)
fig_s, axs_s = plt.subplots(figsize=(10,6))

#palette
palette = sns.color_palette("viridis", len(labels_ls)).as_hex() 
axs_s.set_prop_cycle(color=palette)

#offset: comment line with off = 0 if you need to use an offset
off = np.arange(start = 0, stop = len(spectra[0,:])) * 0.02
off = 0

#plotting spectra
axs_s.plot(wvl, spectra + off, label = labels_ls)

#y and x label
axs_s.set_ylabel('Reflectance', fontsize=14)
axs_s.set_xlabel('Wavelength ('+r'$\mu$m)', fontsize=14)

#x tick spacing
tick_spacing_major = 0.2
tick_spacing_minor = 0.1
axs_s.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing_major)) #big ticks
axs_s.xaxis.set_minor_locator(ticker.MultipleLocator(tick_spacing_minor)) #secondary ticks

plt.legend(bbox_to_anchor=(1.12,0.5), loc='center', ncol=1, reverse = True) #show legend next to the plot, in 1 col with reversed labels
plt.tight_layout()
plt.show()


Using matplotlib backend: MacOSX


# Smoothing (boxcar) - optional
Run to smooth spectra before computing spectral indexes. \
Smoothing helps reducing noise spikes, which in our case heavily affect in the 1.0 um band. 

In [ ]:
from IDL_smooth import smooth #IDL smoothing

spectra_smoothed = np.zeros_like(spectra) #create a new array to contain smoothed spectra

for i in range (0,len(spectra_smoothed[0,:])):
    spectra_smoothed[:,i] = smooth(spectra[:,i],13) #smooth(data, w) -> data = array, w = window size (must be odd); e.g. w = 3

In [ ]:
#use %matplotlib to display figures on screen 
%matplotlib 

#plot with or without offset
fig_s, axs_s = plt.subplots(figsize=(10,6))

#palette
palette = sns.color_palette("viridis", len(labels_ls)).as_hex() #https://matplotlib.org/stable/tutorials/colors/colormaps.html
axs_s.set_prop_cycle(color=palette)

#offset: comment line with off = 0 if you need to use an offset
off = np.arange(start = 0, stop = len(spectra[0,:])) * 0.02
off = 0

#plotting spectra
axs_s.plot(wvl, spectra + off, color='red') #plot the original spectra to see the effect of smoothing
axs_s.plot(wvl, spectra_smoothed + off, label = labels_ls)

#y, x labels
axs_s.set_ylabel('Reflectance', fontsize=10)
axs_s.set_xlabel('Wavelength ('+r'$\mu$m)', fontsize=10)

#x tick spacing
tick_spacing_major = 0.2
tick_spacing_minor = 0.1
axs_s.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing_major)) #big ticks
axs_s.xaxis.set_minor_locator(ticker.MultipleLocator(tick_spacing_minor)) #secondary ticks

plt.title('Spectra, smoothed')
plt.legend(bbox_to_anchor=(1.12,0.5), loc='center', ncol=1, reverse = True) #show legend next to the plot, in 1 col with reversed labels
plt.tight_layout()
plt.show()

Using matplotlib backend: MacOSX


In [41]:
#---- WARNING ----#
# only run this cell if you want to compute the spectral parameters on smoothed spectra
spectra = spectra_smoothed

# Continuum removal

## Function definition 

In [4]:
def find_line (left_sh, right_sh, spectra, axs):

    '''input: 
    left_sh = left shoulder position (in microns)
    right_sh = right shoulder position (in microns)
    spectra = array with spectra
    axs = ax variable to overplot continuum removal line on the spectra

    output: 
    m = angular coefficient of the line (coefficiente angolare)
    q = y-intercept (intercetta/termine noto)
    axs 
    '''

    ls_index = find_nearest(left_sh, wvl) #left shoulder index
    rs_index = find_nearest (right_sh, wvl) #right shoulder index

    YL = spectra[ls_index,:] #left shoulder y coordinate (wvl) for all the spectra 
    YR = spectra[rs_index,:] #right shoulder y coordinate (wvl) for all the spectra 

    length = len(YL) #length = number of spectra 

    XL = np.full(length, wvl[ls_index]) #left shoulder x coordinate (wvl) - (the array is filled all with the same number)
    XR = np.full(length, wvl[rs_index]) #right shoulder x coordinate (wvl) - (the array is filled all with the same number)

    #overplot chosen left and right shoulder position 
    axs.scatter(XL, YL, color='red', s=15) 
    axs.scatter(XR, YR, color='red', s=15)

    #plot a line connecting the shoulders
    axs.plot((XL, XR), (YL, YR), 'r-')
    plt.show()

    #find the equation of the line passing through the shoulders
    #y=mx+q

    m = (YR-YL)/(XR-XL)
    q = YL-m*XL


    return (m,q)


def continuum_removal(left_sh, right_sh, spectra, m, q):


    '''input: 
    left_sh = left shoulder position (in microns) - float
    right_sh = right shoulder position (in microns) - float
    spectra = array with spectra
    m = angular coefficient of the line (coefficiente angolare) - array
    q = y-intercept (intercetta/termine noto) - array

    output: 
    y_cr = continuum removed reflectance values - array

    also plots the continuum removed bands
    
    '''


    #retrieve shoulder indexes
    ls_index = find_nearest(left_sh, wvl) #left shoulder 
    rs_index = find_nearest (right_sh, wvl) #right shoulder 

    #remove the continuum

    Y = np.transpose(spectra[ls_index:rs_index+1,:]) #array containing the original reflectance values, in the selected range, transposed
    X = wvl[ls_index:rs_index+1].reshape(1,-1) #array containing the wavelengths in the selected range, transposed
    M = m[:,np.newaxis] #get a (9,1) array from the original (9,) of m
    Q = q[:,np.newaxis] #get a (9,1) array from the original (9,) of q

 
    y_cr = Y / (M*X+Q)
    
    
    #plot
    fig, axs = plt.subplots(figsize=(10,6)) #figure and axes
    axs.set_prop_cycle(color=palette) #set color 
    axs.plot(wvl[ls_index:rs_index+1], np.transpose(y_cr), label = labels_ls) #plot
    #axs.legend(bbox_to_anchor=(1.1,0.5), loc='center', ncol=1, reverse = True) #show legend next to the plot, in 1 col with reversed/not reversed labels
    axs.set_ylabel('Reflectance (continuum removed)')
    axs.set_xlabel('Wavelength ('+r'$\mu$m)')
    plt.xlim([wvl[ls_index], wvl[rs_index]])


    plt.show()

    return(np.transpose(y_cr))



## Band-to-band CR

In [ ]:
#Ls_arr = np.array([0.816, 1.29, 1.82, 2.25, 2.3595]) #left shoulder's array for SM1200H 
#Rs_arr = np.array([1.172, 1.68, 2.18, 2.346, 2.406]) #right shoulder's array for SM1200H 

#Ls_arr = np.array([0.427, 0.4718, 0.5887, 0.7666, 1.311, 1.8174, 2.1836, 2.2584]) #left shoulder's array for NAu1
#Rs_arr = np.array([0.4706, 0.5641, 0.7501, 1.297, 1.6383, 2.1265, 2.2247, 2.3212]) #right shoulder's array for NAu1

#Ls_arr = np.array([0.4889, 0.5803, 0.809, 1.357, 1.837, 2.2258]) #left shoulder's array for NAu2  
#Rs_arr = np.array([0.5494, 0.7713, 1.308, 1.644, 2.1458, 2.3317]) #right shoulder's array for NAu2

Ls_arr = np.array([0.840, 1.313, 1.835]) #left shoulder's array for hexa 
Rs_arr = np.array([1.15, 1.835, 2.244]) #right shoulder's array for hexa


cr_spectrum_zeros = np.ones_like(spectra) #creating an array containing the y_cr (cr reflectance) at their respective range of wvl and zeros in all the other positions
y_bc = []
ba = []

#computing Y band centre, band area, band depth

for i, value in enumerate(Ls_arr):
    m,q = find_line(Ls_arr[i], Rs_arr[i], spectra, axs_s)
    y_cr = continuum_removal(Ls_arr[i], Rs_arr[i], spectra, m, q) #cr spectra
    cr_spectrum_zeros[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]=y_cr

    y_bc.append(np.min(y_cr, axis = 0)) #list of array where every array has the minima of the spectra for a specific band

    x_cr = wvl[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]
    a1 = np.trapz(y_cr, x_cr, axis = 0) #area below the curve
    a2 = np.max(x_cr)-np.min(x_cr) #rectangle of base x_cr and height=1
    ba.append(a2-a1)


#turning the list into an array
y_bc = np.asarray(y_bc) #band centre (y)
ba = np.asarray(ba) # band area
bd = 1.0 - y_bc #band depth


In [6]:
#X band centre
x_bc= np.zeros_like(y_bc) #same as np.shape(y_bc) = (n,9); n = number of absorption features

for i in range (0,len(y_bc[:,0])): #5
    for j in range(0,len(y_bc[0,:])): #9
        x_bc[i,j] = wvl[find_nearest(y_bc[i,j], cr_spectrum_zeros[:,j])]

In [7]:
#save in a formatted txt file 
#add +'_smoothed' in the case you are saving the smoothed data
np.savetxt('band_centre_binary_'+endmember1_name, np.transpose(x_bc), header = 'Columns = band centre values for each band; rows = samples\nSample type = '+endmember1_name+'(endmember 1) +'+endmember2_name+'(endmember 2); starting at % 90 endmember 1 and % 10 endmember 2', fmt = '%.8f')
np.savetxt('band_depth_binary_'+endmember1_name, np.transpose(bd), header = 'Columns = band depth values for each band; rows = samples\nSample type = '+endmember1_name+'(endmember 1)+'+endmember2_name+'(endmember 2); starting at % 90 endmember 1 and % 10 endmember 2', fmt = '%.8f')
np.savetxt('band_area_binary_'+endmember1_name, np.transpose(ba), header = 'Columns = band area values for each band, computed with the trapezoidal rule; rows = samples\nSample type = '+endmember1_name+'(endmember 1)+'+endmember2_name+'(endmember 2); starting at % 90 endmember 1 and % 10 endmember 2', fmt = '%.8f')

# Reading, plotting and continuum removal of endmembers

In [8]:
#reading endmembers

#read endmembers:
path = 'Data/' 

endm = 'FV7' #change endmember here

s0 = np.loadtxt(path+endm+'_00000.asd.rts.txt', delimiter='\t', skiprows = 1)
s1 = np.loadtxt(path+endm+'_00001.asd.rts.txt', delimiter='\t', skiprows = 1)
s2 = np.loadtxt(path+endm+'_00002.asd.rts.txt', delimiter='\t', skiprows = 1)

wvl = s0[:,0]*0.001 #wavelength in microns

spec = (s0[:,1]+s1[:,1]+s2[:,1])/3

In [ ]:
#---- WARNING ----#
# only run this cell if you want to compute the spectral parameters on smoothed spectra

#IDL smoothing 
spec= smooth(spec, 13) #smooth(data, w) -> data = array, w = window size (must be odd); e.g. w = 3

In [9]:

#use %matplotlib to display figures on screen 
%matplotlib 

#plot with or without offset
fig_s, axs_s = plt.subplots(figsize=(10,6))

#spectra
axs_s.plot(wvl, spec, label = endm)

#y, x label
axs_s.set_ylabel('Reflectance', fontsize=10)
axs_s.set_xlabel('Wavelength ('+r'$\mu$m)', fontsize=10)

#x tick spacing
tick_spacing_major = 0.2
tick_spacing_minor = 0.1
axs_s.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing_major)) #big ticks
axs_s.xaxis.set_minor_locator(ticker.MultipleLocator(tick_spacing_minor)) #secondary ticks

plt.legend() 
plt.tight_layout()
plt.show()

Using matplotlib backend: MacOSX


In [10]:

#Ls_arr = np.array([1.29, 1.82, 2.25, 2.3595]) #left shoulder's array for SM1200H 
#Rs_arr = np.array([1.68, 2.18, 2.346, 2.406]) #right shoulder's array for SM1200H 

#Ls_arr = np.array([0.427, 0.4718, 0.5887, 0.7666, 1.311, 1.8174, 2.1836, 2.2584]) #left shoulder's array for Nau1
#Rs_arr = np.array([0.4706, 0.5641, 0.7501, 1.297, 1.6383, 2.1265, 2.2247, 2.3212]) #right shoulder's array for Nau1

#Ls_arr = np.array([0.4889, 0.5803, 0.809, 1.357, 1.837, 2.2258]) #left shoulder's array for Nau2  
#Rs_arr = np.array([0.5494, 0.7713, 1.308, 1.644, 2.1643, 2.3317]) #right shoulder's array for Nau2* *right shoulder of the 1.9 band is shifted a bit with respect to mix

#Ls_arr = np.array([1.300, 1.835, 2.33]) #left shoulder's array for hexa 
#Rs_arr = np.array([1.835, 2.244, 2.45]) #right shoulder's array for hexa

Ls_arr = np.array ([0.810]) #left shoulder for Fv7
Rs_arr = np.array ([1.209]) #right shoulder for Fv7

cr_spectrum_zeros = np.ones_like(spec) #creating an array containing the y_cr (cr reflectance) at their respective range of wvl and zeros in all the other positions
y_bc = []
ba = []

#computing Y band centre, band area, band depth

for i, value in enumerate(Ls_arr):

    ls_index = find_nearest(Ls_arr[i], wvl) #left shoulder index
    rs_index = find_nearest (Rs_arr[i], wvl) #right shoulder index

    YL = spec[ls_index] #left shoulder y coordinate (wvl) 
    YR = spec[rs_index] #right shoulder y coordinate (wvl) 

    XL = wvl[ls_index] #left shoulder x coordinate (wvl) 
    XR = wvl[rs_index] #right shoulder x coordinate (wvl)

    #overplot chosen left and right shoulder position 
    axs_s.scatter(XL, YL, color='red', s=15) 
    axs_s.scatter(XR, YR, color='red', s=15)

    #plot a line connecting the shoulders
    axs_s.plot((XL, XR), (YL, YR), 'r-')
    plt.show()

    #find the equation of the line passing through the shoulders
    #y=mx+q

    m = (YR-YL)/(XR-XL)
    q = YL-m*XL


    Y = spec[ls_index:rs_index+1] #array containing the original reflectance values, in the selected range, transposed
    X = wvl[ls_index:rs_index+1] #array containing the wavelengths in the selected range, transposed

 
    y_cr = Y / (m*X+q)
    
    
    #plot the lines
    fig, axs = plt.subplots(figsize=(10,6)) #figure and axes
    #axs.set_prop_cycle(color=palette) #set color 
    axs.plot(wvl[ls_index:rs_index+1], y_cr) #plot

    plt.show()

    #continuum removal 
    cr_spectrum_zeros[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]=y_cr
    #BC, BA
    y_bc.append(np.min(y_cr)) #list of array where every array has the minima of the spectra for a specific band
    x_cr = wvl[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]
    a1 = np.trapz(y_cr, x_cr, axis = 0) #area below the curve
    a2 = np.max(x_cr)-np.min(x_cr) #rectangle of base x_cr and height=1
    ba.append(a2-a1)


#turning the list into an array
y_bc = np.asarray(y_bc) #band centre (y)
ba = np.asarray(ba) # band area
bd = 1.0 - y_bc #band depth

x_bc = np.zeros_like(y_bc) #same as np.shape(y_bc) 

#X band centre
for i, value in enumerate(y_bc):
    x_bc[i] = wvl[find_nearest(y_bc[i], cr_spectrum_zeros)]


In [11]:
#print
print('band centre: \n', x_bc) #rows = bands, columns = spectra
print('band depth: \n', bd.round(4))
print('band area: \n', ba.round(4))

band centre: 
 [1.024]
band depth: 
 [0.0866]
band area: 
 [0.0142]


In [12]:
#save in a formatted txt file 
np.savetxt(endm+'_band_centre', x_bc, header = 'Columns = band centre values', fmt = '%.8f')
np.savetxt(endm+'_band_depth', bd, header = 'Columns = band depth values', fmt = '%.8f')
np.savetxt(endm+'_band_area', ba, header = 'Columns = band area values', fmt = '%.8f')